In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=9216  )])

In [2]:
from datasets import load_dataset
import numpy as np
        
raw_dataset = load_dataset("csv", data_files="dataset/Rest_Mex_2022_Sentiment_Analysis_Track_Train.csv", delimiter=";")

Using custom data configuration default-13c2fcae2abc0c4d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/javilonso/.cache/huggingface/datasets/csv/default-13c2fcae2abc0c4d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
print(raw_dataset)
raw_datasets = raw_dataset.shuffle(seed=50)
raw_datasets = raw_datasets["train"].train_test_split(0.3)
print(raw_datasets)
#raw_datasets.save_to_disk("./data/")

DatasetDict({
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction'],
        num_rows: 30212
    })
})
DatasetDict({
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction'],
        num_rows: 21148
    })
    test: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction'],
        num_rows: 9064
    })
})


In [4]:
datapd = raw_dataset["train"].to_pandas()

In [13]:
datapd["Opinion"].tolist()[0]

'Piensen dos veces antes de ir a este hotel, te molestan por todo, mi hijo de seis años traía un inflable de tamaño pequeño y nos decían que era muy grande ahí lo veían llorando y aún así seguían molestando, traíamos una hielera que parece bolsa nada de grande o con ruedas o de ese estilo y Tmb molestándonos las bebidas carísimas y ni agua te dejan tener en la alberca y no hay ni huéspedes en el hotel esto es en el mayan palace el grand mayan pude ser un poco mejor pero la verdad jamás vuelvo a venir a este lugar, te invitan a una cena show de mil pesos por persona y la cena es hoy dogs, las habitaciones nada fuera de lo normal'

In [5]:
#Statistics
print(f'Polarity values({datapd.Polarity.count()}):\n{datapd.Polarity.value_counts()}\n')
print(f'Atraction values({datapd.Attraction.count()}):\n{datapd.Attraction.value_counts()}\n')
print(f'Title values:\n{datapd.Title.str.len().describe()}\n')
print(f'Opinion values:\n{datapd.Opinion.str.len().describe()}') 

Polarity values(30212):
5    20936
4     5878
3     2121
2      730
1      547
Name: Polarity, dtype: int64

Atraction values(30212):
Hotel         16565
Restaurant     8450
Attractive     5197
Name: Attraction, dtype: int64

Title values:
count    30211.000000
mean        25.579987
std         16.099598
min          1.000000
25%         15.000000
50%         22.000000
75%         32.000000
max        153.000000
Name: Title, dtype: float64

Opinion values:
count    30210.000000
mean       625.505859
std        951.186641
min         25.000000
25%        213.000000
50%        309.000000
75%        615.000000
max      20438.000000
Name: Opinion, dtype: float64


In [14]:
# filter non empty title
datapd = datapd[datapd.Title.str.len() > 0]
# filter non empty opinion
datapd = datapd[datapd.Opinion.str.len() > 0]
print(datapd)

                                       Title  \
0                               Pésimo lugar   
1                  No vayas a lugar de Eddie   
2               Mala relación calidad-precio   
3           Minusválido? ¡No te alojes aquí!   
4      Es una porqueria no pierdan su tiempo   
...                                      ...   
30207          Verdadera joya arquitectónica   
30208                              Romántico   
30209                     Parece un castillo   
30210                             Imperdible   
30211                       Muy bonita vista   

                                                 Opinion  Polarity  Attraction  
0      Piensen dos veces antes de ir a este hotel, te...         1       Hotel  
1      Cuatro de nosotros fuimos recientemente a Eddi...         1  Restaurant  
2      seguiré corta y simple: limpieza\n- bad. Tengo...         1       Hotel  
3      Al reservar un hotel con multipropiedad Mayan ...         1       Hotel  
4      No pierdan 

In [15]:
lengths = datapd.Title.str.len()
argmax = np.where(lengths == lengths.min())[0]
print(argmax)
print(datapd.iloc[argmax])

[  120  1441  1575  2856  2899  9616 11103 13342 16188 17490 17887 18271
 19821 22035 24418 25674 26298]
      Title                                            Opinion  Polarity  \
120       😠  Pésima atención, muy groseros!!! Te reciben pé...         1   
1441      🤔  La noche de ayer fuimos mi esposo y yo a cenar...         3   
1575      X  Les falta variedad y sabor en las comidas, en ...         3   
2856      …  Fue un poco interesante pero no es la gran cos...         3   
2899      X  Guanajuato tiene muchos callejones llenos de h...         3   
9617      A  excelente comida, de verdad q los platillos so...         5   
11104     😜  Todos son muy amables, me la pasé excelente, a...         5   
13343     👌  Excelente su grupo de animación. Buscan la mej...         5   
16189     .  Si te quedas en Vidanta Nuevo Vallarta te reco...         5   
17491     .  Excelente servicio y totalmente recomendable (...         5   
17888     😊  me pareció muy buen servicio atendidos por el 

In [20]:
data = datapd.to_json(orient='index', path_or_buf="datapad.json")
